In [7]:
from pathlib import Path
import os
import tifffile
import czifile
from tqdm import tqdm
import napari
import pandas as pd
import numpy as np


In [3]:
# Define the directory containing the raw .czi files and annotations
directory_path = Path("./apeer_annotations/")

# Initialize an empty list to store the .czi and ome.tiff annotation filepaths
czi_images = []
annotations = []

# Iterate through the files in the directory
for file_path in directory_path.glob('*.czi'):
    # Append the file to the corresponding group
    czi_images.append(str(file_path))

# Iterate through the files in the directory
for file_path in directory_path.glob('*.tiff'):
    # Append the file to the corresponding group
    annotations.append(str(file_path))


In [4]:
# Define output directory for .tiff converted .czi files  
processed_czi_dir = "./preprocessed_annotations/images"

# Check if the output folder exists, create it if not
try:
    os.makedirs(processed_czi_dir)
    print(f"Output folder created: {processed_czi_dir}")
except FileExistsError:
    print(f"Output folder already exists: {processed_czi_dir}")
    
for filepath in tqdm(czi_images):
    # Get the filename without the extension
    filename = Path(filepath).stem

    # Read the .czi file and remove singleton dimensions
    img = czifile.imread(filepath)
    img = img.squeeze()

    # Save the image as a regular .tiff file under preprocessed_annotations/images
    tifffile.imwrite(os.path.join(processed_czi_dir, f"{filename}.tiff"), img)


Output folder created: ./preprocessed_annotations/images


100%|██████████| 119/119 [00:01<00:00, 102.64it/s]


In [5]:
# Define output directory for renamed annotations
processed_annotations = "./preprocessed_annotations/annotations"

# Check if the output folder exists, create it if not
try:
    os.makedirs(processed_annotations)
    print(f"Output folder created: {processed_annotations}")
except FileExistsError:
    print(f"Output folder already exists: {processed_annotations}")
    
for filepath in tqdm(annotations):
    # Get the filename without the extension
    filename = Path(filepath).stem
    # Remove _finalprediction.ome so it matches the original image names
    filename = filename.replace("_finalprediction.ome", "")
    
    # Read the ome.tiff file and remove singleton dimensions, ignore ome
    img = tifffile.imread(filepath, is_ome=False)
    
    # Save the image as a regular .tiff file under preprocessed_annotations/images
    tifffile.imwrite(os.path.join(processed_annotations, f"{filename}.tiff"), img)

Output folder created: ./preprocessed_annotations/annotations


100%|██████████| 119/119 [00:02<00:00, 54.31it/s]


At this point each annotations file is a .tiff file where each instance is assigned a pixel value ranging from 1 to the max number of instances. I want to assign each instance its corresponding class id as pixel values ("0 - background, 1 - Dead, 2 - Differentiated, 3 - Undifferentiated") so I can transform these multinary images into YOLO txt based annotations later on.

In [14]:
# Initialize Napari viewer instance
label = tifffile.imread("./preprocessed_annotations/annotations/MIN_P1A01.tiff", is_ome = False)

viewer = napari.Viewer(ndisplay=2)
viewer.add_labels(label, name=filename)


<Labels layer 'MIN_stack_D06' at 0x26fbf4978e0>

In [3]:
# Read measurements_default.csv containing each object_id's class_name
df = pd.read_csv("./apeer_annotations/measurements_default.csv")

In [4]:
df[df["file_name"]=="MIN_StackC01"]

,file_name,class_name,time,z,well_id,object_id,area,area_um2,bbox_y_start,bbox_y_start_um,...,filled_area,filled_area_um2,major_axis_length,major_axis_length_um,minor_axis_length,minor_axis_length_um,orientation,perimeter,perimeter_um,solidity
0,MIN_StackC01,Dead,1,1,A1,1,428,7.998418e+15,13,5.619834e+07,...,428,7.998418e+15,24.715452,1.068437e+08,22.219819,9.605516e+07,-0.675140,76.183766,3.293386e+08,0.957494
1,MIN_StackC01,Dead,1,1,A1,2,529,9.885895e+15,133,5.749523e+08,...,529,9.885895e+15,26.578549,1.148977e+08,25.479090,1.101448e+08,-1.520972,85.012193,3.675034e+08,0.958333
2,MIN_StackC01,Dead,1,1,A1,3,783,1.463262e+16,274,1.184488e+09,...,783,1.463262e+16,36.811543,1.591344e+08,28.224519,1.220132e+08,0.191509,112.325902,4.855792e+08,0.932143
3,MIN_StackC01,Dead,1,1,A1,4,361,6.746329e+15,76,3.285442e+08,...,361,6.746329e+15,25.020680,1.081631e+08,18.514458,8.003707e+07,0.494603,70.526912,3.048843e+08,0.960106
4,MIN_StackC01,Dead,1,1,A1,5,5328,9.956909e+16,14,6.052129e+07,...,5328,9.956909e+16,97.666795,4.222086e+08,78.028768,3.373144e+08,-1.023491,374.333044,1.618223e+09,0.805564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,MIN_StackC01,Undifferentiated,1,1,A1,151,2261,4.225333e+16,986,4.262428e+09,...,2261,4.225333e+16,57.911816,2.503499e+08,49.923290,2.158159e+08,0.080339,178.267027,7.706394e+08,0.977941
327,MIN_StackC01,Undifferentiated,1,1,A1,152,1047,1.956622e+16,1015,4.387794e+09,...,1047,1.956622e+16,41.808507,1.807361e+08,32.830582,1.419249e+08,-1.148928,125.012193,5.404214e+08,0.973953
328,MIN_StackC01,Undifferentiated,1,1,A1,153,3622,6.768755e+16,854,3.691799e+09,...,3622,6.768755e+16,79.375525,3.431364e+08,65.354702,2.825251e+08,-1.016277,270.509668,1.169400e+09,0.847846
329,MIN_StackC01,Undifferentiated,1,1,A1,154,547,1.022228e+16,997,4.309981e+09,...,547,1.022228e+16,29.254213,1.264645e+08,24.425435,1.055899e+08,0.857198,89.840620,3.883765e+08,0.939863


TAKE A LOOK AT THIS NUMPY CODE TO MAKE IT WORK BUT ALMOST THERE

In [8]:
# Load your TIFF file
tiff_file_path = './preprocessed_annotations/annotations/MIN_P1A01.tiff'
tiff_data = tifffile.imread(tiff_file_path)

# Create a mapping for class names to pixel values
class_mapping = {'Dead': 1, 'Differentiated': 2, 'Undifferentiated': 3}

# Map class names to pixel values
pixel_values = np.array([class_mapping[class_name] for class_name in df['class_name']])

# Update pixel values in the TIFF data
tiff_data[tiff_data != 0] = pixel_values[tiff_data[tiff_data != 0] - 1]

In [9]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_labels(tiff_data, name="test")

<Labels layer 'test' at 0x291dd92bd60>

In [10]:
tiff_file_path = './preprocessed_annotations/images/MIN_P1A01.tiff'
tiff_data = tifffile.imread(tiff_file_path)
viewer.add_image(tiff_data, name="test_rawimage")

<Image layer 'test_rawimage' at 0x291de8d0490>